## Classifying digits with convolutional neural networks

This notebook contains the solution to the MNIST activity. 

#### Load the data

Both Keras and TF-Learn contain the MNIST dataset that can be quickly loaded with some helper functions. This solution will use TF-Learn but the Keras solution will be commented out. The two libraries are very similar. 

In [18]:
import numpy as np

import keras
from keras.datasets import mnist

# Load data from TF-Learn 
(X_train, y_train), (X_test, y_test) = mnist.load_data()


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

print(X_train.shape, y_train.shape)

(60000, 28, 28) (60000, 10)


#### Build the ConvNet 

Create a small convolutional that will run on a CPU, so only use about 6 and 8 kernels in each convolutional layer. For the fully connected layer, just use a 32 to 64 units as well. We won't get state of the art performance but we don't want to wait all day for it to run. 

In [10]:

# -------------------------------------------------------------------------------------------
# Keras
import keras
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Dense, Activation, Dropout, Flatten, merge
from keras.layers.convolutional import Conv2D, MaxPooling2D

keras.callbacks.TensorBoard(log_dir='/tmp/tflearn_logs', histogram_freq=0, write_graph=True, write_images=False)


cnn_input = Input(shape=(28, 28, 1), name='Input')

net = Conv2D(6, 3,3, activation='relu')(cnn_input)
net = MaxPooling2D(pool_size=(2,2))(net)

net = Conv2D(8, 3,3, activation='relu')(net)
net = MaxPooling2D(pool_size=(2,2))(net)

# keras uses a flatten layer when going from convolutional layers to normal
net = Flatten()(net)

net = Dense(32, activation='relu')(net)
net = Dropout(p=0.5)(net)

out = Dense(10, activation='softmax')(net)
model = Model(input=cnn_input, output=out)
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 26, 26, 6)         60        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 6)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 11, 11, 8)         440       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 8)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                6432      
__________

C:\Users\fmuel\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (3, 3), activation="relu")`
C:\Users\fmuel\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu")`
C:\Users\fmuel\Anaconda3\lib\site-packages\ipykernel\__main__.py:25: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`
C:\Users\fmuel\Anaconda3\lib\site-packages\ipykernel\__main__.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("In..., outputs=Tensor("de...)`


#### Train the network

Train the network and use the test data as the validation set. 

In [ ]:

# when training the tf-learn model go to http://192.168.1.27:6006

# Keras Train
model.fit(np.expand_dims(X_train, -1), y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 11s - loss: 0.6717 - acc: 0.7664    
Epoch 2/5
60000/60000 [==============================] - 11s - loss: 0.4591 - acc: 0.8443    
Epoch 3/5
60000/60000 [==============================] - 11s - loss: 0.3737 - acc: 0.8745    
Epoch 4/5
60000/60000 [==============================] - 11s - loss: 0.3244 - acc: 0.8892    
Epoch 5/5
44832/60000 [=====================>........] - ETA: 2s - loss: 0.2953 - acc: 0.9007